In [37]:
import boto
import boto.s3.connection
import boto3
import tweepy as tweepy
import requests as requests
import json
import re
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from rake_nltk import Rake
import math
from random import randint
from time import sleep

keys={}
with open("/Users/jagluck/Documents/GitHub/new-artists-twitter-bot/keys.json","r") as f:
    keys = json.loads(f.read())
    
# Consumer keys and access tokens, used for OAuth
tw_consumer_key = keys["tw_consumer_key"]
tw_consumer_secret = keys["tw_consumer_secret"]
tw_access_token = keys["tw_access_token"]
tw_access_token_secret = keys["tw_access_token_secret"]
db_access_key = keys["db_access_key"]
db_secret_key = keys["db_secret_key"]
setlist_api_key = keys['setlist_api_key']

# bucket_name = 'trump-last-tweets' # replace with your bucket name
# object_key = 'tweets.json' # replace with your object key

# emojis={}
# with open("/Users/jagluck/Documents/GitHub/new-artists-twitter-bot/artists.json","r") as f:
#     emojis = json.loads(f.read())

 
# OAuth process, using the keys and tokens
# auth = tweepy.OAuthHandler(tw_consumer_key, tw_consumer_secret)
# auth.set_access_token(tw_access_token, tw_access_token_secret)
 
# Creation of the actual interface, using authentication
# api = tweepy.API(auth)

# s3 = boto3.resource('s3',
#          aws_access_key_id=db_access_key,
#          aws_secret_access_key=db_secret_key)
# my_bucket = s3.Bucket(bucket_name)

# def sendTweet(content):
#     api.update_status(content)
    
# def buildTweet(text, link):
#     tweet = text + " " + link
#     sendTweet(tweet)

# def readDatabase():

#     with open('tweets.json','wb') as data:
#         my_bucket.download_fileobj(object_key, data)

#     old_tweets={}
#     with open("C:\\Users\\jagluck\\Documents\\Github\\twitter-user-emoji-bot\\tweets.json","r") as f:
#         text = f.read()
    
#     old_tweets = json.loads(text)

#     # return old_tweets['tweets']
#     return old_tweets

# def updateDatabase(json_data):

#     with open('tweets.json', 'wb') as outfile:
#         outfile.write(json_data)

#     with open('tweets.json', 'rb') as data:
#         my_bucket.upload_fileobj(data, object_key)

def sendReqSet(num, id):
    
    url = 'https://api.setlist.fm/rest/1.0/venue/' + str(id) + '/setlists/?p=' + str(num)
    headers = {'Accept': 'application/json', 'x-api-key': setlist_api_key}
    r = ''
    
    while r == '':
        # try to get ID
        try:
            r = requests.get(url, headers=headers)
        # if max tries exceed, sleep and try again
        except:
            time.sleep(5)
            
    page = r.json()
    return page

# get the artists out of a response from the api
def getVenueArtists(venueResp):
    artistsFromPage = set()
    for artistSet in venueResp['setlist']:
        artistsFromPage = artistsFromPage.union(artistSet['artist']['name'])
    return artistsFromPage

def getAllArtistsFromVenue(venueId):
    
    artistsFromVenue = set()
    page = 330
    
    # get first result page
    venueResp = sendReqSet(page, venueId)
    
    # pull responses untl you get all of them
    while('setlist' in venueResp):
        
        # sleep random time
        sleep((randint(500,1000)/1000))
        
        # get results from page
        artistsFromPage = getVenueArtists(venueResp)
        artistsFromVenue = artistsFromVenue.union(artistsFromPage)
        
        # set up next loop
        page = page + 1
        venueResp = sendReqSet(page, venueId)
    
    return artistsFromVenue

def getAllArtistsFromCity(venueIds):
    artistsFromCity = set()
    
    for venueId in venueIds:
        artistsFromVenue = getAllArtistsFromVenue(venueId)
        artistsFromCity = artistsFromCity.union(artistsFromVenue)
    
    return artistsFromVenue
    

def getCurrentArtists():
    return []


def runBot():
#     old_tweets = readDatabase()
    current = getCurrentArtists()
#     new_tweets = current['tweets']
#     links = current['links']

#     for new in new_tweets:
#         if new not in old_tweets:
#             translated = translateText(new)
#             tweet_id = links[new]
#             print(translated)
            #sendTweet(translated + " " + "https://twitter.com/realDonaldTrump/status/" + str(tweet_id))

#     json_data = json.dumps(new_tweets)
#     updateDatabase(json_data.encode('utf-8'))


# runBot()

In [38]:
venueIds = ['3d635a7']
artistsFromCity = getAllArtistsFromCity(venueIds)

In [39]:
print(artistsFromCity)

{'.', 'W', 'C', 'c', 'F', 's', 'r', 'o', 'K', 'B', 'b', 'l', 'V', 'H', 'E', ' ', 'N', 'e', 'I', 'Y', 'x', 'G', 'f', 'T', 'd', 'k', 't', 'h', 'n', 'R', 'A', 'a', 'U', 'M', 'g', 'i', 'P', 'D', 'S', 'p', 'J', '!', 'm', 'y', '&', 'v', 'u', 'L'}
